In [7]:
import pandas as pd
from tqdm import tqdm

In [4]:
# preprocess json first
from datasets import load_dataset

In [5]:
ds = load_dataset("dim/tldr_17_50k")

Generating train split: 100%|██████████| 50000/50000 [00:00<00:00, 50863.55 examples/s]


In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
        num_rows: 50000
    })
})

In [26]:
import re

def parse_age_gender(text):
    pattern = r'\b(\d{1,3})\s*(M|F|NB)\b'
    result = re.findall(pattern, text, re.IGNORECASE)
    result = [(int(age), gender) for age,gender in result if int(age) < 100 and int(age) > 10]
    return result
# Example usage:
title = "I (24M) entered my first relationship (22F). My friend (28NB) thinks it's normal. Another friend (19 NB) disagrees."

result = parse_age_gender(title)
print(result)

[(24, 'M'), (22, 'F'), (28, 'NB'), (19, 'NB')]


In [27]:
ds_with_gender = {
    "text": [],
    "age": [],
    "gender": []
}


for i in tqdm(range(len(ds["train"]))):
    text = ds["train"][i]["normalizedBody"]
    

    possible_age_gender = parse_age_gender(text)
    
    if possible_age_gender != []:
        age = possible_age_gender[0][0]
        gender = possible_age_gender[0][1]

        ds_with_gender["text"].append(text)
        ds_with_gender["age"].append(age)
        ds_with_gender["gender"].append(gender)

print(len(ds_with_gender))


100%|██████████| 50000/50000 [00:08<00:00, 6046.04it/s]

3


In [28]:
df = pd.DataFrame(ds_with_gender)
df[100:200].to_csv("./check_age_gender.csv", index=False)

In [35]:
df_labeled = pd.read_csv("./check_age_gender_regex_labeled.csv", sep=";")
print(df_labeled.Correct.sum() / len(df_labeled) * 100)
df_labeled.head(1)

54.0


,text,age,gender,Correct,Unnamed: 4
0,I'll call the person I'm interested in Jane [1...,16,F,False,0


In [36]:
df_labeled = pd.read_csv("./check_age_gender_phi3_labeled.csv", sep=";")
print(df_labeled.Correct.sum() / len(df_labeled) * 100)
df_labeled.head(1)

91.0


,text,age,gender,Correct
0,I'll call the person I'm interested in Jane [1...,-1,M,True
